## Equal Weight Screener
In this we'll be weighting each company in the S&P 500 equally

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [13]:
symbol =  'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'ALSNEENAGD(T/R KTSA OEMGBLSQ )LCA', 'calculationPrice': 'close', 'open': 141.4, 'openTime': 1675719769485, 'openSource': 'flaicofi', 'close': 133.91, 'closeTime': 1686799018809, 'closeSource': 'afolfici', 'high': 141.5, 'highTime': 1674346471787, 'highSource': 'etprac enud  il5y1eimde', 'low': 135.27, 'lowTime': 1656239378792, 'lowSource': 'deudee epainci l1 mtyr5', 'latestPrice': 134.67, 'latestSource': 'Close', 'latestTime': 'January 29, 2021', 'latestUpdate': 1627004022521, 'latestVolume': 185737426, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 132.37, 'delayedPriceTime': 1642681781073, 'oddLotDelayedPrice': 136.87, 'oddLotDelayedPriceTime': 1662573658018, 'extendedPrice': 133.98, 'extendedChange': -0.18, 'extendedChangePercent': -0.00141, 'extendedPriceTime': 1655886957889, 'previousClose': 138.26, 'previousVolume': 146235479, 'change': -5.2, 'changePercent': -0.03757, '

In [27]:
columns = ['Ticker', 'Stock Price (USD)', 'Market Capitalization','Number of Shares to Buy']
finalDataframe = pd.DataFrame(columns = columns)

,Ticker,Stock Price (USD),Market Capitalization,Number of Shares to Buy


In [38]:
finalDataframe = pd.DataFrame(columns = columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    finalDataframe = finalDataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = columns
        ),
    ignore_index = True
    )
#endfor

In [39]:
finalDataframe

,Ticker,Stock Price (USD),Market Capitalization,Number of Shares to Buy
0,A,122.27,37162240337,N/A
1,AAL,17.63,10836828512,N/A
2,AAP,150.18,10152171266,N/A
3,AAPL,138.39,2312338866250,N/A
4,ABBV,103.71,189335356276,N/A
...,...,...,...,...
500,YUM,104.41,31847709451,N/A
501,ZBH,157.18,31957545495,N/A
502,ZBRA,400.62,21468189500,N/A
503,ZION,45.84,7250211902,N/A


In [125]:
def chunks(lst, n):
    """Yield successive n-sized chunks from list"""
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [126]:
symbolGroups = list(chunks(stocks['Ticker'],100))
symbolStrings = []
for i in range(0,len(symbolGroups)):
    symbolStrings.append(','.join(symbolGroups[i]))
finalDataframe = pd.DataFrame(columns=columns)

for symbolString in symbolStrings:
    batchApiCallUrl = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbolString}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchApiCallUrl).json()
    for symbol in symbolString.split(','):
        finalDataframe = finalDataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = columns
            ),
            ignore_index = True
        )
finalDataframe

,Ticker,Stock Price (USD),Market Capitalization,Number of Shares to Buy
0,A,121.07,36985821272,N/A
1,AAL,17.36,10542738127,N/A
2,AAP,153.30,10456714057,N/A
3,AAPL,133.93,2248931208337,N/A
4,ABBV,104.07,185553464391,N/A
...,...,...,...,...
500,YUM,104.21,31714851484,N/A
501,ZBH,154.81,31955853162,N/A
502,ZBRA,393.65,20842480597,N/A
503,ZION,44.18,7321670251,N/A


In [127]:
portfolioSize = float(input('Enter value of your portfolio:'))
try:
    val = float(portfolioSize)
    print(val)
except ValueError:
    print("That's not a number.\nPlease try again.")
    print('Please enter an integer.')

Enter value of your portfolio:10000000
10000000.0


In [146]:
positionSize = val/len(finalDataframe.index)
for i in range(0, len(finalDataframe.index)):
    finalDataframe.loc[i, 'Number of Shares to Buy'] = math.floor(portfolioSize / finalDataframe.loc[i, 'Stock Price (USD)'])

In [147]:
writer = pd.ExcelWriter('recommended trades.xlsx',engine = 'xlsxwriter')
finalDataframe.to_excel(writer, "Recommended Trades", index = False)

In [148]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

In [149]:
#writer.sheets['Recommended Trades'].set_column('A:A',18, string_format)
#writer.sheets['Recommended Trades'].set_column('B:B',18, string_format)
#writer.sheets['Recommended Trades'].set_column('C:C',18, string_format)
#writer.sheets['Recommended Trades'].set_column('D:D',18, string_format)
#writer.save()

writer.sheets['Recommended Trades'].write('A1','Ticker',string_format)
writer.sheets['Recommended Trades'].write('B1','Stock Price (USD)',dollar_format)
writer.sheets['Recommended Trades'].write('C1','Market Capitalization',dollar_format)
writer.sheets['Recommended Trades'].write('D1','Number of Shares to Buy',integer_format)

0

In [150]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price (USD)', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'C': ['Number of Share to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18,column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0],column_formats[column][1])
writer.save()